In [1]:
import time
import json
from openai import OpenAI
from dotenv import dotenv_values


In [2]:
env = dotenv_values()

In [3]:
client = OpenAI(api_key=env['OPENAI_API_KEY'])
assistant_id = "asst_VdhqgK4Hy3hwkUdr6NBSsas7"


# File upload

In [229]:
files_cursor = client.files.list(limit=100)
files = [file for file in files_cursor]

for file in files:
  print(file.id, file.filename)


file-VYEU7NsGqim4K6T9KRLreS f9fb2ac7-a262-4650-be40-8cf2e46a8fdc.pdf
file-5e9px6frkpHDopN6VWFQSw f9fb2ac7-a262-4650-be40-8cf2e46a8fdc.pdf
file-ATMpjNtUmctSy14DVqCPTz 0c0fcd32-e3a5-4585-a5f7-7f347c787d97.pdf


In [230]:
for file in files:
  res = client.files.delete(file.id)
  print(res)


FileDeleted(id='file-VYEU7NsGqim4K6T9KRLreS', deleted=True, object='file')
FileDeleted(id='file-5e9px6frkpHDopN6VWFQSw', deleted=True, object='file')
FileDeleted(id='file-ATMpjNtUmctSy14DVqCPTz', deleted=True, object='file')


# Messages


In [192]:
messages_cursor = client.beta.threads.messages.list(thread_id)
messages = [message for message in messages_cursor]

for message in messages:
  print(message.id, message.content[0].text.value)


msg_eRFSTD20yUgBVeN2bTf3xqXG The document is a comprehensive introduction to mathematical logic, aimed primarily at first-year students at the Slovak Technical University in Bratislava. It outlines the fundamental concepts and techniques of both propositional and predicate logic, including the construction and interpretation of logical formulas. The text is organized into chapters that cover various topics, such as truth tables, tautologies, formal systems, semantic trees, and the resolution method, along with applications in electronics and neural networks. It elaborates on the Gentzen calculus, a syntactic method utilizing sequents for proof derivation, and delves into modal and many-valued logics, indicating different types of modal logic, including alethic, epistemic, temporal, and deontic. Each chapter serves as a lecture on its own, structured to facilitate self-study and includes numerous exercises to solidify understanding. The authors emphasize the necessity of a basic underst

In [193]:
for message in messages:
  client.beta.threads.messages.delete(
    message_id=message.id,
    thread_id=thread_id
  )


# Pipeline

In [4]:
# prompt_content = "Summarize the provided document into a single paragraph of about 300 words, in the primary language that is most used throughout the document. Do not use any numbered lists or bullet points, do not cite sources. Return just plain text."

prompt_content = "Summarize the provided document into a single paragraph of about 250 to 350 words, in the primary language that is most used throughout the document. Do not use any numbered lists or bullet points, do not cite sources. Return just plain text. Talk only about the contents, not the document itself."

def get_summary(filename):
  thread = client.beta.threads.create()
  
  file = client.files.create(
    file=open(f"../../dataset/{filename}", "rb"),
    purpose="assistants",
  )
  
  client.beta.threads.messages.create(
    thread_id=thread.id,
    content=prompt_content,
    attachments=[{
      "file_id": file.id,
      "tools": [{ "type": "file_search" }]
    }],
    role="user"
  )

  run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant_id,
  )

  if run.status != "completed":
    raise Exception("Run failed:", run.status)

  messages_cursor = client.beta.threads.messages.list(thread.id)
  messages = [message for message in messages_cursor]
  message = messages[0]

  response_content = message.content[0].text.value

  client.beta.threads.delete(thread.id)
  client.files.delete(file.id)

  return response_content


In [7]:
with open("../../../dataset/_merged.json") as f:
  documents = json.load(f)
  documents = [documents[0]]

print(documents)

[{'pk': '8288e199-5927-4eb1-8c40-07387287264f', 'filename': '8288e199-5927-4eb1-8c40-07387287264f.pdf', 'title': 'Odporúčanie pre softvérových inžinierov'}]


In [236]:
with open("../../dataset/_merged.json") as f:
  documents = json.load(f)
  documents = [documents[0]]

results = []

for i, document in enumerate(documents):
  title = document['title']
  filename = document['filename']
  summary = get_summary(filename)

  print(f"[{i + 1}/{len(documents)}]: {title}")
  print(summary)
  print

  results.append({
    "id": document['pk'],
    "title": document['title'],
    "summary": summary,
  })

  time.sleep(65)


[1/1]: Odporúčanie pre softvérových inžinierov
Odporúčacie systémy v softvérovom inžinierstve (OSSI) sa zameriavajú na poskytovanie podpory vývojárom pri rôznych činnostiach, vrátane znovupoužitia kódu a písania efektívnych hlásení o chybách. Tieto systémy vychádzajú zo štúdia zdrojového kódu a jeho príbuzných artefaktov a poskytujú konkrétne odporúčania týkajúce sa používania programovacích rozhraní, knižníc a aplikačných rámcov. Proces vývoja týchto systémov obsahuje štyri hlavné fázy: analýzu, návrh, implementáciu a validáciu, pričom dôležité rozhodnutia sú rozdelené do dvoch skupín, a to tie týkajúce sa metódy odporúčania a interakcie so systémom. Existujú rôzne prístupy k odporúčaniu, z ktorých najčastejšie je filtrovanie na základe obsahu a kolaboratívne filtrovanie, ktoré závisí od údajov získaných z histórie interakcií používateľov. OSI vyžaduje vytvorenie modelu používateľa na presné prispôsobenie odporúčaní aktuálneho a relevantného kódu. Príklady úspešných existujúcich systé

KeyboardInterrupt: 

In [ ]:
with open('01_summaries.json', 'w') as f:
  json.dump(results, f, indent=2, ensure_ascii=False)
